<a href="https://colab.research.google.com/github/vivekvision/PySparkDataWrangling/blob/main/MultiDimensionalSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is not Reommended approach, its just for illustration. Recommended practice is described in anothere notebook: **MultiDimentionalData.ipynb**

In [7]:
!pip install pyspark

In [8]:
import pandas as pd
import pyspark.sql.functions as sf

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("testApp")\
        .getOrCreate()

In [9]:
# free pyspark package can not read excel file directly 
df = pd.read_excel("./MultiDimensionalSample.xlsx", sheet_name="Sheet1", header=None, dtype=str)

In [10]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NaN,NaN,NaN,NaN,FY2020,FY2020,FY2020,FY2020,FY2020,FY2020,FY2020,FY2020,FY2020
1,NaN,NaN,NaN,NaN,December,December,December,November,November,November,October,October,October
2,NaN,NaN,NaN,NaN,TotalCases,OverRun,Budget,TotalCases,OverRun,Budget,TotalCases,OverRun,Budget
3,NaN,NaN,NaN,NaN,YTD,YTD,YTD,YTD,YTD,YTD,YTD,YTD,YTD
4,Asia,Singapore,Sub A,Item 1,34,65,45,30,60,34,30,60,34


Above data is required in a noraml data-frame table format 




In [12]:
# Convert pandas data-frame to Spark data-frame
s_df = spark.createDataFrame(df.astype(str))

In [15]:
s_df.show(4)

+---+---+---+---+----------+--------+--------+----------+--------+--------+----------+-------+-------+-------+
|  0|  1|  2|  3|         4|       5|       6|         7|       8|       9|        10|     11|     12|row_num|
+---+---+---+---+----------+--------+--------+----------+--------+--------+----------+-------+-------+-------+
|nan|nan|nan|nan|    FY2020|  FY2020|  FY2020|    FY2020|  FY2020|  FY2020|    FY2020| FY2020| FY2020|      0|
|nan|nan|nan|nan|  December|December|December|  November|November|November|   October|October|October|      1|
|nan|nan|nan|nan|TotalCases| OverRun|  Budget|TotalCases| OverRun|  Budget|TotalCases|OverRun| Budget|      2|
|nan|nan|nan|nan|       YTD|     YTD|     YTD|       YTD|     YTD|     YTD|       YTD|    YTD|    YTD|      3|
+---+---+---+---+----------+--------+--------+----------+--------+--------+----------+-------+-------+-------+
only showing top 4 rows



In [16]:
# add a serial number / row number 
# pyspark function monotonically_increasing_id() provides row number 
s_df = s_df.withColumn("row_num", sf.monotonically_increasing_id())

In [17]:
s_df.show(4)

+---+---+---+---+----------+--------+--------+----------+--------+--------+----------+-------+-------+-------+
|  0|  1|  2|  3|         4|       5|       6|         7|       8|       9|        10|     11|     12|row_num|
+---+---+---+---+----------+--------+--------+----------+--------+--------+----------+-------+-------+-------+
|nan|nan|nan|nan|    FY2020|  FY2020|  FY2020|    FY2020|  FY2020|  FY2020|    FY2020| FY2020| FY2020|      0|
|nan|nan|nan|nan|  December|December|December|  November|November|November|   October|October|October|      1|
|nan|nan|nan|nan|TotalCases| OverRun|  Budget|TotalCases| OverRun|  Budget|TotalCases|OverRun| Budget|      2|
|nan|nan|nan|nan|       YTD|     YTD|     YTD|       YTD|     YTD|     YTD|       YTD|    YTD|    YTD|      3|
+---+---+---+---+----------+--------+--------+----------+--------+--------+----------+-------+-------+-------+
only showing top 4 rows



In [31]:
# read first 4 rows into a separate data-frame 
header_df = s_df.where(sf.col('row_num') <5)
header_df= header_df.drop('row_num')
header_df.show()

+----+---------+-----+------+----------+--------+--------+----------+--------+--------+----------+-------+-------+
|   0|        1|    2|     3|         4|       5|       6|         7|       8|       9|        10|     11|     12|
+----+---------+-----+------+----------+--------+--------+----------+--------+--------+----------+-------+-------+
| nan|      nan|  nan|   nan|    FY2020|  FY2020|  FY2020|    FY2020|  FY2020|  FY2020|    FY2020| FY2020| FY2020|
| nan|      nan|  nan|   nan|  December|December|December|  November|November|November|   October|October|October|
| nan|      nan|  nan|   nan|TotalCases| OverRun|  Budget|TotalCases| OverRun|  Budget|TotalCases|OverRun| Budget|
| nan|      nan|  nan|   nan|       YTD|     YTD|     YTD|       YTD|     YTD|     YTD|       YTD|    YTD|    YTD|
|Asia|Singapore|Sub A|Item 1|        34|      65|      45|        30|      60|      34|        30|     60|     34|
+----+---------+-----+------+----------+--------+--------+----------+--------+--

In [32]:
# drop first 4 columns
header_df = header_df.drop(header_df.columns[0],header_df.columns[1], header_df.columns[2], header_df.columns[3])
header_df.show()

+----------+--------+--------+----------+--------+--------+----------+-------+-------+
|         4|       5|       6|         7|       8|       9|        10|     11|     12|
+----------+--------+--------+----------+--------+--------+----------+-------+-------+
|    FY2020|  FY2020|  FY2020|    FY2020|  FY2020|  FY2020|    FY2020| FY2020| FY2020|
|  December|December|December|  November|November|November|   October|October|October|
|TotalCases| OverRun|  Budget|TotalCases| OverRun|  Budget|TotalCases|OverRun| Budget|
|       YTD|     YTD|     YTD|       YTD|     YTD|     YTD|       YTD|    YTD|    YTD|
|        34|      65|      45|        30|      60|      34|        30|     60|     34|
+----------+--------+--------+----------+--------+--------+----------+-------+-------+



In [33]:
# Merge rows into single row using pyspark function: collect_list()
exprs = [sf.collect_list(x) for x in header_df.columns]
header_df = header_df.agg(*exprs)
header_df.toPandas()

,collect_list(4),collect_list(5),collect_list(6),collect_list(7),collect_list(8),collect_list(9),collect_list(10),collect_list(11),collect_list(12)
0,"[FY2020, December, TotalCases, YTD, 34]","[FY2020, December, OverRun, YTD, 65]","[FY2020, December, Budget, YTD, 45]","[FY2020, November, TotalCases, YTD, 30]","[FY2020, November, OverRun, YTD, 60]","[FY2020, November, Budget, YTD, 34]","[FY2020, October, TotalCases, YTD, 30]","[FY2020, October, OverRun, YTD, 60]","[FY2020, October, Budget, YTD, 34]"


In [34]:
# retrieve first row from above data-frame
firstRow = header_df.first()

In [35]:
firstRow

Row(collect_list(4)=['FY2020', 'December', 'TotalCases', 'YTD', '34'], collect_list(5)=['FY2020', 'December', 'OverRun', 'YTD', '65'], collect_list(6)=['FY2020', 'December', 'Budget', 'YTD', '45'], collect_list(7)=['FY2020', 'November', 'TotalCases', 'YTD', '30'], collect_list(8)=['FY2020', 'November', 'OverRun', 'YTD', '60'], collect_list(9)=['FY2020', 'November', 'Budget', 'YTD', '34'], collect_list(10)=['FY2020', 'October', 'TotalCases', 'YTD', '30'], collect_list(11)=['FY2020', 'October', 'OverRun', 'YTD', '60'], collect_list(12)=['FY2020', 'October', 'Budget', 'YTD', '34'])

In [53]:
multi_header_List = list(firstRow.asDict().values())



In [54]:
multi_header_List

[['FY2020', 'December', 'TotalCases', 'YTD', '34'],
 ['FY2020', 'December', 'OverRun', 'YTD', '65'],
 ['FY2020', 'December', 'Budget', 'YTD', '45'],
 ['FY2020', 'November', 'TotalCases', 'YTD', '30'],
 ['FY2020', 'November', 'OverRun', 'YTD', '60'],
 ['FY2020', 'November', 'Budget', 'YTD', '34'],
 ['FY2020', 'October', 'TotalCases', 'YTD', '30'],
 ['FY2020', 'October', 'OverRun', 'YTD', '60'],
 ['FY2020', 'October', 'Budget', 'YTD', '34']]

In [55]:
multi_header_List =  ['_'.join(x) for x in multi_header_List]

In [56]:
multi_header_List

['FY2020_December_TotalCases_YTD_34',
 'FY2020_December_OverRun_YTD_65',
 'FY2020_December_Budget_YTD_45',
 'FY2020_November_TotalCases_YTD_30',
 'FY2020_November_OverRun_YTD_60',
 'FY2020_November_Budget_YTD_34',
 'FY2020_October_TotalCases_YTD_30',
 'FY2020_October_OverRun_YTD_60',
 'FY2020_October_Budget_YTD_34']

In [57]:
# concate all the list elements of above list  
columnList = ["Region","Country","Sub", "Item"] + multi_header_List

In [58]:
columnList

['Region',
 'Country',
 'Sub',
 'Item',
 'FY2020_December_TotalCases_YTD_34',
 'FY2020_December_OverRun_YTD_65',
 'FY2020_December_Budget_YTD_45',
 'FY2020_November_TotalCases_YTD_30',
 'FY2020_November_OverRun_YTD_60',
 'FY2020_November_Budget_YTD_34',
 'FY2020_October_TotalCases_YTD_30',
 'FY2020_October_OverRun_YTD_60',
 'FY2020_October_Budget_YTD_34']

In [59]:
# take content/body into separate data-frame
data_df = s_df.where(sf.col('row_num') > 5)
data_df= data_df.drop('row_num')
data_df.show(4)

+----+---------+-----+------+---+---+---+---+---+---+---+---+---+
|   0|        1|    2|     3|  4|  5|  6|  7|  8|  9| 10| 11| 12|
+----+---------+-----+------+---+---+---+---+---+---+---+---+---+
|Asia|Singapore|Sub A|Item 3| 34| 65| 45| 30| 60| 34| 30| 60| 34|
|Asia|Singapore|Sub B|Item 1| 34| 65| 45| 30| 60| 34| 30| 60| 34|
|Asia|Singapore|Sub B|Item 2| 34| 65| 45| 30| 60| 34| 30| 60| 34|
|Asia|Singapore|Sub B|Item 3| 34| 65| 45| 30| 60| 34| 30| 60| 34|
+----+---------+-----+------+---+---+---+---+---+---+---+---+---+
only showing top 4 rows



In [60]:
# change column name for all the columns 
for i in range(0, len(data_df.columns)):
  data_df = data_df.withColumnRenamed(data_df.columns[i], columnList[i])

In [61]:
data_df.columns

['Region',
 'Country',
 'Sub',
 'Item',
 'FY2020_December_TotalCases_YTD_34',
 'FY2020_December_OverRun_YTD_65',
 'FY2020_December_Budget_YTD_45',
 'FY2020_November_TotalCases_YTD_30',
 'FY2020_November_OverRun_YTD_60',
 'FY2020_November_Budget_YTD_34',
 'FY2020_October_TotalCases_YTD_30',
 'FY2020_October_OverRun_YTD_60',
 'FY2020_October_Budget_YTD_34']

In [62]:
data_df.show(4)

+------+---------+-----+------+---------------------------------+------------------------------+-----------------------------+---------------------------------+------------------------------+-----------------------------+--------------------------------+-----------------------------+----------------------------+
|Region|  Country|  Sub|  Item|FY2020_December_TotalCases_YTD_34|FY2020_December_OverRun_YTD_65|FY2020_December_Budget_YTD_45|FY2020_November_TotalCases_YTD_30|FY2020_November_OverRun_YTD_60|FY2020_November_Budget_YTD_34|FY2020_October_TotalCases_YTD_30|FY2020_October_OverRun_YTD_60|FY2020_October_Budget_YTD_34|
+------+---------+-----+------+---------------------------------+------------------------------+-----------------------------+---------------------------------+------------------------------+-----------------------------+--------------------------------+-----------------------------+----------------------------+
|  Asia|Singapore|Sub A|Item 3|                           

In [63]:
# UnPivot the columns
index_cols=["Region", "Country", "Sub", "Item"]
pivot_cols=[c for c in data_df.columns if c not in index_cols]

In [64]:
unPivotExp = "stack("+str(len(pivot_cols))+","+",".join(["'{"+str(i)+"}',{"+str(i)+"}" for i in range(len(pivot_cols))])+") as (Param, Value)"
data_df = data_df.selectExpr(*index_cols, unPivotExp.format(*pivot_cols))

In [65]:
data_df.show(4)

+------+---------+-----+------+--------------------+-----+
|Region|  Country|  Sub|  Item|               Param|Value|
+------+---------+-----+------+--------------------+-----+
|  Asia|Singapore|Sub A|Item 3|FY2020_December_T...|   34|
|  Asia|Singapore|Sub A|Item 3|FY2020_December_O...|   65|
|  Asia|Singapore|Sub A|Item 3|FY2020_December_B...|   45|
|  Asia|Singapore|Sub A|Item 3|FY2020_November_T...|   30|
+------+---------+-----+------+--------------------+-----+
only showing top 4 rows



In [66]:
data_df = data_df.withColumn('Year', sf.split(sf.col('Param'),'_').getItem(0))


In [67]:
data_df.show(4)

+------+---------+-----+------+--------------------+-----+------+
|Region|  Country|  Sub|  Item|               Param|Value|  Year|
+------+---------+-----+------+--------------------+-----+------+
|  Asia|Singapore|Sub A|Item 3|FY2020_December_T...|   34|FY2020|
|  Asia|Singapore|Sub A|Item 3|FY2020_December_O...|   65|FY2020|
|  Asia|Singapore|Sub A|Item 3|FY2020_December_B...|   45|FY2020|
|  Asia|Singapore|Sub A|Item 3|FY2020_November_T...|   30|FY2020|
+------+---------+-----+------+--------------------+-----+------+
only showing top 4 rows



In [68]:
data_df = data_df.withColumn('Month', sf.split(sf.col('Param'),'_').getItem(1))
data_df = data_df.withColumn('Metrics', sf.split(sf.col('Param'),'_').getItem(2))
data_df = data_df.withColumn('ComulativeFreq', sf.split(sf.col('Param'),'_').getItem(3))

In [69]:
data_df = data_df.drop('Param')

In [70]:
data_df = data_df.withColumnRenamed('Value', 'Measure')

In [71]:
data_df.show()

+------+---------+-----+------+-------+------+--------+----------+--------------+
|Region|  Country|  Sub|  Item|Measure|  Year|   Month|   Metrics|ComulativeFreq|
+------+---------+-----+------+-------+------+--------+----------+--------------+
|  Asia|Singapore|Sub A|Item 3|     34|FY2020|December|TotalCases|           YTD|
|  Asia|Singapore|Sub A|Item 3|     65|FY2020|December|   OverRun|           YTD|
|  Asia|Singapore|Sub A|Item 3|     45|FY2020|December|    Budget|           YTD|
|  Asia|Singapore|Sub A|Item 3|     30|FY2020|November|TotalCases|           YTD|
|  Asia|Singapore|Sub A|Item 3|     60|FY2020|November|   OverRun|           YTD|
|  Asia|Singapore|Sub A|Item 3|     34|FY2020|November|    Budget|           YTD|
|  Asia|Singapore|Sub A|Item 3|     30|FY2020| October|TotalCases|           YTD|
|  Asia|Singapore|Sub A|Item 3|     60|FY2020| October|   OverRun|           YTD|
|  Asia|Singapore|Sub A|Item 3|     34|FY2020| October|    Budget|           YTD|
|  Asia|Singapor